Note: this notebook is the same as the original, just used it to generate equilbrium caches in parallel



### Generate equilibrium cache

In [1]:
import math
from simtk import unit
from openmmtools import testsystems, states, mcmc, multistate
import os
import tempfile
import pickle
from perses.annihilation.lambda_protocol import RelativeAlchemicalState
import mdtraj as md
import numpy as np
from simtk.unit.quantity import Quantity
from matplotlib import pyplot as plt

DEBUG:matplotlib:$HOME=/home/zhangi
DEBUG:matplotlib:CONFIGDIR=/home/zhangi/.config/matplotlib
DEBUG:matplotlib:matplotlib data path: /home/zhangi/miniconda3/envs/perses-sims/lib/python3.7/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:loaded rc file /home/zhangi/miniconda3/envs/perses-sims/lib/python3.7/site-packages/matplotlib/mpl-data/matplotlibrc
DEBUG:matplotlib:matplotlib version 3.1.1
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', 'zipimport', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_abc', '_bootlocale', '_locale', 'site', 'os', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'types', 'importlib', 'importlib._bootstrap', 'importlib._bootstrap_

DEBUG:matplotlib:CACHEDIR=/home/zhangi/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/zhangi/.cache/matplotlib/fontlist-v310.json
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [3]:
htf = pickle.load(open("/data/chodera/zhangi/perses_benchmark/neq/7/7/7_vacuum.pickle", "rb" ))


In [4]:
htf.hybrid_system.usesPeriodicBoundaryConditions()


False

In [5]:
n_replicas = 12  # Number of temperature replicas.
T_min = 298.0 * unit.kelvin  # Minimum temperature.
T_max = 600.0 * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(n_replicas-1)) - 1.0) / (math.e - 1.0)
                for i in range(n_replicas)]


In [6]:
temperatures


[Quantity(value=298.0, unit=kelvin),
 Quantity(value=314.72669195828234, unit=kelvin),
 Quantity(value=333.0452538211905, unit=kelvin),
 Quantity(value=353.107182955901, unit=kelvin),
 Quantity(value=375.0783946491671, unit=kelvin),
 Quantity(value=399.14059425264895, unit=kelvin),
 Quantity(value=425.49277991454665, unit=kelvin),
 Quantity(value=454.35288832536025, unit=kelvin),
 Quantity(value=485.9595970883531, unit=kelvin),
 Quantity(value=520.5742986206017, unit=kelvin),
 Quantity(value=558.4832619091029, unit=kelvin),
 Quantity(value=600.0, unit=kelvin)]

In [7]:
alchemical_state = RelativeAlchemicalState.from_system(htf.hybrid_system)
alchemical_state.set_alchemical_parameters(1)


In [8]:
thermodynamic_states = [states.ThermodynamicState(system=htf.hybrid_system, temperature=T)
                        for T in temperatures]
compound_states = [states.CompoundThermodynamicState(thermodynamic_state=thermodynamic_state, composable_states=[alchemical_state])
                        for thermodynamic_state in thermodynamic_states]

In [9]:
compound_states[0].n_particles


50

In [10]:
thermodynamic_states[0].is_periodic


False

In [11]:
move = mcmc.GHMCMove(timestep=4.0*unit.femtoseconds, n_steps=250)
simulation = multistate.ReplicaExchangeSampler(mcmc_moves=move, number_of_iterations=1000)


In [12]:
reporter_file = "/data/chodera/zhangi/perses_benchmark/neq/7/7/7_vacuum_phe_1ns.nc"
reporter = multistate.MultiStateReporter(reporter_file, checkpoint_interval=1)
simulation.create(thermodynamic_states=compound_states,
                  sampler_states=states.SamplerState(htf.hybrid_positions),
                  storage=reporter)
simulation.run()

DEBUG:openmmtools.multistate.multistatereporter:Initial checkpoint file automatically chosen as /data/chodera/zhangi/perses_benchmark/neq/7/7/7_vacuum_phe_1ns_checkpoint.nc
DEBUG:mpiplus.mpiplus:Cannot find MPI environment. MPI disabled.
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.storage_exists of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2b794d8dde80>>
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._display_citations at 0x2b792885eea0>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._display_citations at 0x2b792881c378>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._initialize_reporter at 0x2b792881c620>
DEBUG:openmmtools.multistate.multistatereporter:Serialized state thermodynamic_states/0 is  11219B | 10.956KB | 0.011MB
DEBUG:openmmtools.utils:Storing thermodynamic states took    0.047s
DEBUG:openmmtools.multistate.multistatesampler:St

Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys., 135:194110, 2011. DOI:10.1063/

DEBUG:openmmtools.utils:Computing energy matrix took    0.940s
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.write_energies of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2b794d8dde80>>
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:openmmtools.multistate.multistatesampler:Iteration 1/1000
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2b792885eb70>
DEBUG:openmmtools.multistate.replicaexchange:Mixing replicas...
DEBUG:openmmtools.utils:Mixing of replicas took    0.008s
DEBUG:openmmtools.multistate.replicaexchange:Accepted 41472/41472 attempted swaps (100.0%)
DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _pr

DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _propagate_replica serially.
DEBUG:mpiplus.mpiplus:Running _get_replica_move_statistics serially.
DEBUG:openmmtools.utils:Propagating all replicas took   10.877s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.156s
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration at 0x2b792881c730>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration_items at 0x2b792881c9d8>
DEBUG:openmmtools.utils:Storing sampler states took    0.006s
DEBUG:openmmtools.utils:Writing iteration information to storage took    0.029s
DEBUG:openmmtools.multistate.multistatesampler:Not enough iterations for online analysis (self.online_analysis_minimum_iterations = 200)
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._online_analysis at 0x2b79

DEBUG:openmmtools.multistate.multistatesampler:Iteration 8/1000
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2b792885eb70>
DEBUG:openmmtools.multistate.replicaexchange:Mixing replicas...
DEBUG:openmmtools.utils:Mixing of replicas took    0.001s
DEBUG:openmmtools.multistate.replicaexchange:Accepted 11512/41472 attempted swaps (27.8%)
DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _propagate_replica serially.
DEBUG:mpiplus.mpiplus:Running _get_replica_move_statistics serially.
DEBUG:openmmtools.utils:Propagating all replicas took   15.457s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.157s
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteratio

...

DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration_items at 0x2b792881c9d8>
DEBUG:openmmtools.utils:Storing sampler states took    0.002s
DEBUG:openmmtools.utils:Writing iteration information to storage took    0.024s
DEBUG:openmmtools.multistate.multistatesampler:Not enough iterations for online analysis (self.online_analysis_minimum_iterations = 200)
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._online_analysis at 0x2b792881e510>
DEBUG:openmmtools.multistate.multistatesampler:*** ONLINE analysis free energies:
DEBUG:openmmtools.multistate.multistatesampler:        -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.2   -28.4
DEBUG:openmmtools.utils:Computing online free energy estimate took    0.009s
DEBUG:openmmtools.multistate.multistatesampler:Iteration took 9.765s.
DEBUG:openmmtools.multistate.multistatesampler:Estimated completion in 2:57:14.950682, at Tue Aug  4 13:10:33 2020 (con

DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _propagate_replica serially.
DEBUG:mpiplus.mpiplus:Running _get_replica_move_statistics serially.
DEBUG:openmmtools.utils:Propagating all replicas took   10.619s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.160s
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration at 0x2b792881c730>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration_items at 0x2b792881c9d8>
DEBUG:openmmtools.utils:Storing sampler states took    0.003s
DEBUG:openmmtools.utils:Writing iteration information to storage took    0.029s
DEBUG:openmmtools.multistate.multistatesampler:Not enough iterations for online analysis (self.online_analysis_minimum_iterations = 200)
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._online_analysis at 0x2b79

DEBUG:openmmtools.multistate.multistatesampler:Iteration 23/1000
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2b792885eb70>
DEBUG:openmmtools.multistate.replicaexchange:Mixing replicas...
DEBUG:openmmtools.utils:Mixing of replicas took    0.001s
DEBUG:openmmtools.multistate.replicaexchange:Accepted 11160/41472 attempted swaps (26.9%)
DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _propagate_replica serially.
DEBUG:mpiplus.mpiplus:Running _get_replica_move_statistics serially.
DEBUG:openmmtools.utils:Propagating all replicas took   13.355s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.244s
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iterati

DEBUG:openmmtools.multistate.multistatesampler:        -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.2   -34.5
DEBUG:openmmtools.utils:Computing online free energy estimate took    0.008s
DEBUG:openmmtools.multistate.multistatesampler:Iteration took 11.207s.
DEBUG:openmmtools.multistate.multistatesampler:Estimated completion in 3:01:16.599543, at Tue Aug  4 13:16:43 2020 (consuming total wall clock time 3:06:06.939983).
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:openmmtools.multistate.multistatesampler:Iteration 27/1000
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2b792885eb70>
DEBUG:openmmtools.multistate.replicaexchange:Mixing replicas...
DEBUG:openmmtools.utils:Mixing of replicas 

SystemError: <built-in function Context_getState> returned a result with an error set